In [ ]:
import copy
import matplotlib
import pandas
import warnings
import requests
import scipy
from scipy import stats
import subprocess
import numpy as np
import numpy
from matplotlib import pyplot as plt
import json
import seaborn
import networkx as nx
import curatorbin
import os

In [ ]:
os.makedirs("task_data", exist_ok=True)

In [ ]:
task_id = "sys_perf_linux_3_node_replSet_last_continuous_fcv_mixed_writes_replica_60e0443449d43bd027e482583ff414a963aa01c5_21_08_05_19_22_39"
metric_name = "MixedWrites.W3InsertOne.1"
task_record = requests.get(f"https://cedar.mongodb.com/rest/v1/perf/task_id/{task_id}").json()
tests = [t for t in task_record if metric_name in t["info"]["test_name"]]
if len(tests) == 0:
    raise ValueError("The metric requested does not exist")

metric_record = tests[0]

In [ ]:
def get_data(result):
    ftdc_file = f"./task_data/{metric_name}.ftdc"
    csv_file = f"./task_data/{metric_name}.csv"
    try:
        df = pandas.read_csv(f"./task_data/{metric_name}.csv")
    except FileNotFoundError:
        if not os.path.isfile(ftdc_file):
            r = requests.get(result["artifacts"][0]["download_url"])
            with open(ftdc_file, "wb") as f:
                f.write(r.content)
        curatorbin.run_curator("ftdc", "export", "csv", "--input", ftdc_file, "--output", csv_file)
        df = pandas.read_csv(csv_file)
    return df

In [ ]:
data = get_data(metric_record)

In [ ]:
for field in ["counters.n", "counters.ops", "counters.size", "counters.errors", "timers.dur", "timers.total"]:
    data[field] = data[field].diff().fillna(data[field])
for field in ["timers.dur", "timers.total"]:
    data[field] = data[field] / 1e+9
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    data["ts"] = pandas.to_datetime(data["ts"])
    data["ts"] = data["ts"].dt.to_period("1s") # Sample the timestamps down to 1 second granularity.

In [ ]:
b = pandas.DataFrame()
b["throughput"] = data.groupby("ts").count()["id"]
b["latency"] = data.groupby("ts").mean()["timers.dur"]
b["latency-median"] = data.groupby("ts").median()["timers.dur"]
b["latency-max"] = data.groupby("ts").max()["timers.dur"]

In [ ]:
b.plot.area(figsize=(20,12), subplots=True)